In [1]:
%reload_ext autoreload
%autoreload 2

setup_already_done = False

In [2]:
import sys
import os

In [3]:
IN_COLAB = 'google.colab' in sys.modules
IN_COLAB

False

In [4]:
if not setup_already_done:
    if(IN_COLAB):
        !git clone https://github.com/Silver0x10/VideoPrediction_MovingMNIST.git
        %cd VideoPrediction_MovingMNIST
    else:
        %cd ..
    if(not os.path.exists("data/mnist_test_seq.npy")):
        if(not os.path.exists("data")): 
            %mkdir data
        %cd data
        !wget -q https://www.cs.toronto.edu/~nitish/unsupervised_video/mnist_test_seq.npy
        %cd ..
        
    %pip install -qr requirements.txt  --quiet
    setup_already_done = True

/home/silverio/Projects/uni/DeepLearning/VideoPrediction_MovingMNIST
Note: you may need to restart the kernel to use updated packages.


In [5]:
# import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
# from torch import optim, nn, utils, Tensor
# import torchvision
# from torchvision.transforms import ToTensor
import lightning.pytorch as pl

/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 0.1.43ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.1build1 is an invalid version and will not be supported in a future release
  warnings.warn(


In [6]:
# !nvidia-smi

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [8]:
from src.MovingMNIST import MovingMNIST

dataset = MovingMNIST(data_path="data/mnist_test_seq.npy")

# Visualization Test
dataset.visualize(0)


In [9]:
loader = DataLoader(dataset)

In [10]:
from src.simpleLSTM import simpleLSTM

model = simpleLSTM()


In [11]:
for data in loader:
    print(model.training_step(data,0))
    break

tensor(2893.5442, grad_fn=<MseLossBackward0>)


In [ ]:
# trainer = pl.Trainer(limit_train_batches=1, max_epochs=1, accelerator="cpu")
# trainer = pl.Trainer(max_epochs=1, accelerator="cpu")
# trainer = pl.Trainer(max_epochs=1, accelerator="gpu")
trainer = pl.Trainer(max_epochs=1)

trainer.fit(model=model, train_dataloaders=loader)